In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
train=pd.read_csv("original datasets/train.csv")
test=pd.read_csv("original datasets/test.csv")
train["type"]="train"
test["type"]="test"

In [3]:
d=pd.concat([train,test])

In [4]:
galaxies=d.galaxy.unique().tolist()

In [5]:
frames=[]
for galaxy in galaxies:
    df = d.loc[d.galaxy==galaxy, :]
    df=df.sort_values('galactic year')
    index=df.index
    df.set_index(df['galactic year'], inplace=True)
    for i in range(2, 79):
        nna =df.iloc[:,[i]].notnull().sum()
        nna = nna.tolist()
        if (nna[0]>1):
            df.iloc[:,[i]] = df.iloc[:,[i]].interpolate(method='slinear')
    df.set_index(index, inplace=True)
    frames.append(df)
d_inter=pd.concat(frames)

In [6]:
dr=d_inter.iloc[:, 2:79]

In [7]:
normalized_dr=(dr-dr.min())/(dr.max()-dr.min())

In [8]:
imputer = KNNImputer(n_neighbors=5, weights='distance')
filled=pd.DataFrame(imputer.fit_transform(normalized_dr), index = normalized_dr.index, columns = normalized_dr.columns)

In [9]:
denormalized_dr = (filled*(dr.max()-dr.min()))+dr.min()

In [10]:
d_inter.iloc[:,2:79]=denormalized_dr

In [11]:
train_p=d_inter.loc[d_inter.type=="train", :].iloc[:, 0:80]
train_p=train_p.sort_index()

In [12]:
test_p=d_inter.loc[d_inter.type=="test",:].iloc[:, 0:79]
test_p=test_p.sort_index()

In [13]:
train_p.to_csv("datasets/combined_interpol_knn3/train.csv", index=False)
test_p.to_csv("datasets/combined_interpol_knn3/test.csv", index=False)